# TODO

1. NdBSpline: a tuple of non-zero b-spline vectors
2. *[DONE]* Tests with 2D
3. *[DONE]* and 3D
4. *[DONE]* vector of `x`   + tests
5. *[DONE]* trailing dimensions of `c`: reshape, loop, reshape back + tests
6. *[DONE]* test 2D random, non-separable
7. test mixed degrees: k = [2, 3] etc

--

8. get rid of `itertools.product` : iterate over c.flat (subarray!)
9. change np.unravel_index / ravel_multi_index to stride manipulation (pointers!)

In [20]:
import operator
import itertools

import numpy as np

In [1]:
def B(x, k, i, t):
    if k == 0:
        return 1.0 if t[i] <= x < t[i+1] else 0.0
    if t[i+k] == t[i]:
        c1 = 0.0
    else:
        c1 = (x - t[i])/(t[i+k] - t[i]) * B(x, k-1, i, t)
    if t[i+k+1] == t[i+1]:
        c2 = 0.0
    else:
        c2 = (t[i+k+1] - x)/(t[i+k+1] - t[i+1]) * B(x, k-1, i+1, t)
    return c1 + c2

def bspline(x, t, c, k):
    n = len(t) - k - 1
    assert (n >= k+1) and (len(c) >= n)
    return sum(c[i] * B(x, k, i, t) for i in range(n))

In [2]:
def _naive_eval(x, t, c, k):
    """
    Naive B-spline evaluation. Useful only for testing!
    """
    if x == t[k]:
        i = k
    else:
        i = np.searchsorted(t, x) - 1
    assert t[i] <= x <= t[i+1]
    assert i >= k and i < len(t) - k
    return sum(c[i-j] * B(x, k, i-j, t) for j in range(0, k+1))

In [7]:
_naive_eval(1.5, spl.t, spl.c, 3), spl(1.5)

(3.3749999999999996, array(3.375))

In [4]:
import numpy as np
from scipy.interpolate import make_interp_spline

def prod(iterable):
    """
    Product of a sequence of numbers.

    Faster than np.prod for short lists like array shapes, and does
    not overflow if using Python integers.
    """
    product = 1
    for x in iterable:
        product *= x
    return product

In [5]:
x = np.arange(6)
y = x**3
spl = make_interp_spline(x, y, k=3)

In [6]:
spl.t

array([0., 0., 0., 0., 2., 3., 5., 5., 5., 5.])

In [ ]:
spl.c

In [ ]:
y_1 = x**3 + 2*x
spl_1 = make_interp_spline(x, y_1, k=3)
spl_1.t, spl_1.c

In [8]:
# make the outer product of 1D coefficients

c2 = np.outer(spl_1.c, spl.c)

with np.printoptions(suppress=True, linewidth=100):
    print(c2)

NameError: name 'spl_1' is not defined

In [9]:
def bspline2(xy, t, c, k):
    x, y = xy
    tx, ty = t
    nx = len(tx) - k - 1
    assert (nx >= k+1)
    ny = len(ty) - k - 1
    assert (ny >= k+1)
    return sum(c[ix, iy] * B(x, k, ix, tx) * B(y, k, iy, ty) for ix in range(nx) for iy in range(ny))

In [10]:
for x0, y0 in [(1.5, 0.5), (2.5, 1), (0.5, 1.5)]:
    print((x0**3 + 2*x0)* y0**3, ' -- ',
          bspline2((x0, y0), (spl.t, spl_1.t), c2, k=3))

NameError: name 'spl_1' is not defined

In [ ]:
(0.5**3 + 2*0.5) * 1.5**3

In [11]:
class NdBSpline0:
    def __init__(self, t, c, k=3):
        """Tensor product spline object.
        
        c[i1, i2, ..., id] * B(x1, i1) * B(x2, i2) * ... * B(xd, id)
        
        
        Parameters
        ----------
        c : ndarray, shape (n1, n2, ..., nd, ...)
            b-spline coefficients
        t : tuple of 1D ndarrays
            knot vectors in directions 1, 2, ... d
            ``len(t[i]) == n[i] + k + 1``
        k : int or length-d tuple of integers
            spline degrees.
        """
        ndim = len(t)
        assert ndim <= len(c.shape)
        
        try:
            len(k)
        except TypeError:
            # make k a tuple
            k = (k,)*ndim

        self.k = tuple(operator.index(ki) for ki in k)
        self.t = tuple(np.asarray(ti, dtype=float) for ti in t)
        self.c = c

    def __call__(self, x):
        ndim = len(self.t)
        # a single evaluation point: `x` is a 1D array_like, shape (ndim,)
        assert len(x) == ndim
        
        # get the indices in an ndim-dimensional vector
        i = ['none',]*ndim
        for d in range(ndim):
            td, xd = self.t[d], x[d]
            k = self.k[d]
            
            # find the index for x[d]
            if xd == td[k]:
                i[d] = k
            else:
                i[d] = np.searchsorted(td, xd) - 1
            assert td[i[d]] <= xd <= td[i[d]+1]
            assert i[d] >= k and i[d] < len(td) - k
        i = tuple(i)
        
        # iterate over the dimensions, form linear combinations of
        # products B(x_1) * B(x_2) * ... B(x_N) of (k+1)**N b-splines
        # which are non-zero at `i = (i_1, i_2, ..., i_N)`.
        result = 0
        iters = [range(i[d] - self.k[d], i[d] + 1) for d in range(ndim)]
        for idx in itertools.product(*iters):
            term = self.c[idx] * prod(B(x[d], self.k[d], idx[d], self.t[d]) for d in range(ndim))
            result += term
        return result

In [201]:
class NdBSpline:
    """Tensor product spline object.

    c[i1, i2, ..., id] * B(x1, i1) * B(x2, i2) * ... * B(xd, id)


    Parameters
    ----------
    c : ndarray, shape (n1, n2, ..., nd, ...)
        b-spline coefficients
    t : tuple of 1D ndarrays
        knot vectors in directions 1, 2, ... d
        ``len(t[i]) == n[i] + k + 1``
    k : int or length-d tuple of integers
        spline degrees.
    """
    def __init__(self, t, c, k=3):
        ndim = len(t)
        assert ndim <= len(c.shape)
        
        try:
            len(k)
        except TypeError:
            # make k a tuple
            k = (k,)*ndim

        self.k = tuple(operator.index(ki) for ki in k)
        self.t = tuple(np.asarray(ti, dtype=float) for ti in t)
        self.c = c

    def __call__(self, xi):
        """Evaluate the tensor product b-spline at coordinates.
        
        Parameters
        ----------
        xi : array_like, shape(..., ndim)
            The coordinates to evaluate the interpolator at.
            This can be a list or tuple of ndim-dimensional points
            or an array with the shape (num_points, ndim).
            
        Returns
        -------
        values : ndarray, shape xi.shape[:-1] + self.c.shape[ndim:]
            Interpolated values at xi
        """
        ndim = len(self.t)

        # prepare xi : shape (..., m1, ..., md) -> (1, m1, ..., md)
        xi = np.asarray(xi, dtype=float)
        xi_shape = xi.shape
        xi = xi.reshape(-1, xi_shape[-1])        
        xi = np.ascontiguousarray(xi)

        if xi_shape[-1] != ndim:
            raise ValueError(f"Shapes: xi.shape={xi_shape} and ndim={ndim}")
        
        # prepare the coefficients: flatten the trailing dimensions
        c1 = self.c.reshape(self.c.shape[:ndim] + (-1,))
        
        num_c_tr = c1.shape[-1]  # # of trailing coefficients
        out = np.empty(xi.shape[:-1] + (num_c_tr,), dtype=float)
        
        # iterate over the data points
        for j in range(xi.shape[0]):           
            x = xi[j]

            # get the indices in an ndim-dimensional vector
            i = ['none',]*ndim
            b = np.empty((ndim, max(self.k)+1), dtype=float) * np.nan
            for d in range(ndim):
                td, xd = self.t[d], x[d]
                k = self.k[d]

                # find the index for x[d]
                if xd == td[k]:
                    i[d] = k
                else:
                    i[d] = np.searchsorted(td, xd) - 1
                assert td[i[d]] <= xd <= td[i[d]+1]
                assert i[d] >= k and i[d] < len(td) - k

                # (k+1) b-splines which are non-zero at x[d] 
                b[d, :k+1] = [B(xd, k, j, td) for j in range(i[d]-k, i[d]+1)]   
            i = tuple(i)

            # iterate over the dimensions, form linear combinations of
            # products B(x_1) * B(x_2) * ... B(x_N) of (k+1)**N b-splines
            # which are non-zero at `i = (i_1, i_2, ..., i_N)`.
            result = np.zeros(num_c_tr, dtype=float)
            iters = [range(i[d] - self.k[d], i[d] + 1) for d in range(ndim)]
            for idx in itertools.product(*iters):
                factor = prod(b[d, idx[d] - i[d] + self.k[d]] for d in range(ndim))
                # loop over the trailing values of self.c explicitly
                for i_c in range(num_c_tr):
                    result[i_c] += c1[idx + (i_c,)] * factor
    ##                print(idx, i_c)
            
            ### The above is in principle enough for a prototype.
            ### Below, replicate it with flat indexing.
            result_flat = np.zeros(num_c_tr, dtype=float)
            c1r = c1.ravel()
            for iflat in range((k+1)**ndim):
                idx_b = np.unravel_index(iflat, (k+1,)*ndim)
                factor = prod(b[d, idx_b[d]] for d in range(ndim))   # bsplines
        
                # shift the indices into c1.ravel()
                idx_b = list(idx_b)
                for d in range(ndim):
                    idx_b[d] += i[d] - self.k[d] 

                # collect linear combinations of coef * factor
                idx_c = idx_b + ['none',]
                for i_c in range(num_c_tr):
                    idx_c[-1] = i_c
                    idx_cflat = np.ravel_multi_index(tuple(idx_c), c1.shape)
                    result_flat[i_c] += c1r[idx_cflat] * factor

            from numpy.testing import assert_allclose
            assert_allclose(result, result_flat, atol=1e-15)
                
            # copy the result over: this is in fact a loop over num_c_tr
            out[j, ...] = result_flat
            
        return out.reshape(xi_shape[:-1] + self.c.shape[ndim:])

In [14]:
bspl2 = NdBSpline((spl.t, spl_1.t), c2, k=3)

for x0, y0 in [(1.5, 2.5), (2.5, 1), (0.5, 1.5)]:
    print((x0**3 + 2*x0)* y0**3, ' -- ',
          bspline2((x0, y0), (spl.t, spl_1.t), c2, k=3), '--',
         ## naive_B_2((x0, y0), (spl.t, spl_1.t), c2, k=3), )
          bspl2((x0, y0))
         )

NameError: name 'spl_1' is not defined

In [ ]:
bspl2([(1.5, 2.5), (2.5, 1), (0.5, 1.5)])

In [ ]:
spl2 = NdBSpline((spl.t, spl_1.t), c2, k=3)

spl2((1.5, 2.5))

In [ ]:
spl.t, spl_1.t

In [ ]:
k = 3
p = itertools.product(*(range(0, k+1),)*3)
for j, tpl in enumerate(p):
    #print(j, tpl, a[tpl])
    pass

# Tests

In [147]:
def bspline2(xy, t, c, k):
    """A naive 2D tensort product spline evaluation."""
    x, y = xy
    tx, ty = t
    nx = len(tx) - k - 1
    assert (nx >= k+1)
    ny = len(ty) - k - 1
    assert (ny >= k+1)
    return sum(c[ix, iy] * B(x, k, ix, tx) * B(y, k, iy, ty) for ix in range(nx) for iy in range(ny))


def naive_B_2(xy, t, c, k):
    """
    Naive B-spline evaluation, another way.
    """
    x, y = xy
    tx, ty = t
    
    if x == tx[k]:
        ix = k
    else:
        ix = np.searchsorted(tx, x) - 1
    assert tx[ix] <= x <= tx[ix + 1]
    assert ix >= k and ix < len(tx) - k

    if y == ty[k]:
        iy = k
    else:
        iy = np.searchsorted(ty, y) - 1
    assert ty[iy] <= y <= ty[iy + 1]
    assert iy >= k and iy < len(ty) - k
    
    res = sum(c[ix-jx, iy - jy] * B(x, k, ix-jx, tx) * B(y, k, iy-jy, ty)
               for jx in range(0, k+1) for jy in range(0, k+1))
    
    result = 0
    for jx in range(ix-k, ix+1):
        for jy in range(iy-k, iy+1):
            result += c[jx, jy] * B(x, k, jx, tx) * B(y, k, jy, ty)
            
    assert np.allclose(result,  res) #, result-res
    return result

In [148]:
from numpy.testing import assert_allclose
from numpy.random import default_rng


def make_2d_case():
    # make a 2D separable spline
    x = np.arange(6)
    y = x**3
    spl = make_interp_spline(x, y, k=3)
    
    y_1 = x**3 + 2*x
    spl_1 = make_interp_spline(x, y_1, k=3)

    t2 = (spl.t, spl_1.t)
    c2 = np.outer(spl_1.c, spl.c)
    
    return t2, c2, 3

In [202]:
# test 2D

def test_2D_separable():
    xi = [(1.5, 2.5), (2.5, 1), (0.5, 1.5)]  
    t2, c2, k = make_2d_case()
    target = [(x**3 + 2*x) * y**3 for (x, y) in xi]
    
    # sanity check: bspline2 gives the product as constructed
    assert_allclose([bspline2(xy, t2, c2, k) for xy in xi],
                    target,
                    atol=1e-14)

    # check evaluation on a 2D array: the 1D array of 2D points
    bspl2 = NdBSpline(t2, c2, k=3)
    assert bspl2(xi).shape == (len(xi), )
    assert_allclose(bspl2(xi),
                    target, atol=1e-14)
    
    # now check on a multidim xi
    rng = default_rng(12345)
    xi = rng.uniform(size=(4, 3, 2)) * 5
    result = bspl2(xi)
    assert result.shape == (4, 3)
    
    # also check the values
    x, y = xi.reshape((-1, 2)).T
    assert_allclose(result.ravel(),
                    (x**3 + 2*x) * y**3, atol=1e-14)


def test_2D_separable_2():
    # test `c` with trailing dimensions, i.e. c.ndim > ndim
    ndim = 2
    xi = [(1.5, 2.5), (2.5, 1), (0.5, 1.5)]  
    target = [(x**3 + 2*x) * y**3 for (x, y) in xi]

    t2, c2, k = make_2d_case()
    c2_4 = np.dstack((c2, c2, c2, c2))   # c22.shape = (6, 6, 4)
    
    xy = (1.5, 2.5)
    bspl2_4 = NdBSpline(t2, c2_4, k=3)
    result = bspl2_4(xy)
    val_single = NdBSpline(t2, c2, k)(xy)
    assert result.shape == (4,)
    assert_allclose(result,
                    [val_single,]*4, atol=1e-14)
    
    # now try the array xi : the output.shape is (3, 4)
    # where 3 is the number of points in xi and 4 is the trailing dimension of c
    assert bspl2_4(xi).shape == np.shape(xi)[:-1] + bspl2_4.c.shape[ndim:]
    assert_allclose(bspl2_4(xi) - np.asarray(target)[:, None],
                    0, atol=5e-14)
    
    # two trailing dimensions
    c2_22 = c2_4.reshape((6, 6, 2, 2))
    bspl2_22 = NdBSpline(t2, c2_22, k=3)
    
    result = bspl2_22(xy)
    assert result.shape == (2, 2)
    assert_allclose(result,
                    [[val_single, val_single],
                     [val_single, val_single]], atol=1e-14)

    # now try the array xi : the output shape is (3, 2, 2)
    # for 3 points in xi and c trailing dimensions being (2, 2)
    assert bspl2_22(xi).shape == np.shape(xi)[:-1] + bspl2_22.c.shape[ndim:]
    assert_allclose(bspl2_22(xi) - np.asarray(target)[:, None, None],
                    0, atol=5e-14)

def test_2D_random():
    rng = default_rng(12345)
    k = 3
    tx = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=7)) * 3, 3, 3, 3, 3]
    ty = np.r_[0, 0, 0, 0, np.sort(rng.uniform(size=8)) * 4, 3, 4, 4, 4]
    c = rng.uniform(size=(tx.size-k-1, ty.size-k-1))
    
    spl = NdBSpline((tx, ty), c, k=k)
    
    xi = (1., 1.)
    assert_allclose(spl(xi), bspline2(xi, (tx, ty), c, k), atol=1e-14)

    xi = np.c_[[1, 1.5, 2],
               [1.1, 1.6, 2.1]]
    assert_allclose(spl(xi), [bspline2(xy, (tx, ty), c, k) for xy in xi], atol=1e-14)

    
    
#### RUN test ###
test_2D_separable()
test_2D_separable_2()
test_2D_random()

In [203]:
# test 3D

def make_3d_case():
    # make a 3D separable spline
    x = np.arange(6)
    y = x**3
    spl = make_interp_spline(x, y, k=3)
    
    y_1 = x**3 + 2*x
    spl_1 = make_interp_spline(x, y_1, k=3)

    y_2 = x**3 + 3*x + 1
    spl_2 = make_interp_spline(x, y_2, k=3)
    
    t2 = (spl.t, spl_1.t, spl_2.t)
    c2 = spl_2.c[None, None, :] * spl_1.c[None, :, None] * spl.c[:, None, None]
    
    return t2, c2, 3


def test_3D_separable():
    rng = default_rng(12345)
    x, y, z = rng.uniform(size=(3, 11)) * 5
    target = x**3 * (y**3 + 2*y) * (z**3 + 3*z + 1)

    t3, c3, k = make_3d_case()
    bspl3 = NdBSpline(t3, c3, k=3)

    xi = [_ for _ in zip(x, y, z)]
    result = bspl3(xi)
    assert result.shape == (11,)
    assert_allclose(result, target, atol=1e-14)
        
test_3D_separable()

In [170]:
    t3, c3, k = make_3d_case()
    bspl3 = NdBSpline(t3, c3, k=3)

In [172]:
c3.size

216

In [176]:
c3.strides

(288, 48, 8)

In [177]:
c3.ravel().strides

(8,)